In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [3]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)


#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'

In [4]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 10

In [8]:
data_width = np.shape(x_train)[2]
data_height = np.shape(x_train)[1]
print(data_width)
print(data_height)

96
4


In [12]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(132, 4, 96)
(132,)
(87, 4, 96)
(87,)


In [13]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(data_width,data_height)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [14]:
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback()])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 132 arrays: [array([['36.621', '31.250', '27.832', '25.391', '27.832', '34.180',
        '34.180', '25.879', '20.508', '20.508', '23.926', '27.344',
        '19.531', '17.578', '18.555', '20.508', '24.414', '25.8...